In [1]:

import requests
import os
import yaml
from dotenv import load_dotenv


load_dotenv('./.env')
nai_api_key = os.getenv('NAI_API_KEY')

nai_url= 'https://ai.nutanix.com/api/v1/chat/completions'

nai_api_key = nai_api_key.strip()



def send_inference_request(system_prompt, user_prompt):
    inference_input = {
        "model": "vllm-llama-3-1",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        "max_tokens": 1024,
        "stream": False
    }

    response = requests.post(nai_url, 
                            headers={
                                    "Authorization": f"Bearer {nai_api_key}", 
                                    "accept": "application/json",
                                    "Content-Type": "application/json"
                                }, 
                            json=inference_input, 
                            verify=False)
    return response.json()


In [5]:
def find_go_files():
    source_code_file = []
    source_code_manual_unit_test_file = []
    source_code_codegen_unit_test_file = []

    for root, dirs, files in os.walk("./codegen-observation"): 
        for file in files:
            if file.endswith(".go") and not file.endswith("_test.go") and not file.endswith("_codegen_test.go"):
                source_code_file.append(os.path.relpath(os.path.join(root, file)))
            elif file.endswith("_test.go") and not file.endswith("_codegen_test.go"):
                source_code_manual_unit_test_file.append(os.path.relpath(os.path.join(root, file)))
            elif file.endswith("_codegen_test.go"):
                source_code_codegen_unit_test_file.append(os.path.relpath(os.path.join(root, file)))
    
    return source_code_file, source_code_manual_unit_test_file, source_code_codegen_unit_test_file

source_code_file, source_code_manual_unit_test_file, source_code_codegen_unit_test_file = find_go_files()

In [8]:
source_code_manual_unit_test_file

['codegen-observation/piplani/api_key/api_key_test.go',
 'codegen-observation/piplani/inference/inference_test.go',
 'codegen-observation/hritik/cluster/cluster_test.go',
 'codegen-observation/hritik/health/health_test.go',
 'codegen-observation/hritik/catalog/catalog_test.go',
 'codegen-observation/hritik/endpoint/endpoint_test.go',
 'codegen-observation/hritik/client/client_test.go']

In [9]:
with open('Prompts/prompt_01.yaml', 'r') as file:
    prompts = yaml.safe_load(file)
    
for file, manual_ut, codegen_ut in zip(source_code_file, source_code_manual_unit_test_file, source_code_codegen_unit_test_file):
    with open(file, 'r') as source_file:
        go_source_code = source_file.read()

    with open(manual_ut, 'r') as test_file:
        go_unit_test_code = test_file.read()

    with open(codegen_ut, 'r') as test_file:
        go_unit_test_code = test_file.read()
        
    system_prompt = prompts.get('system_prommpt','')

    user_prompt = prompts.get('user_prompt', '')
    user_prompt = user_prompt.replace('<GO_SOURCE_CODE>', go_source_code).replace('<GO_MANUAL_UNIT_TEST_CODE>', manual_ut).replace('<GO_COPILOT_UNIT_TEST_CODE>',codegen_ut)
    
    response_from_model = send_inference_request(system_prompt=system_prompt, user_prompt=user_prompt)["choices"][0]["message"]["content"]

    output_dir = os.path.dirname(f'./Responses/{file[:-3]}_UTs_comparison.txt')
    os.makedirs(output_dir, exist_ok=True)
    with open(f'./Responses/{file[:-3]}_UTs_comparison.txt', 'w') as file:
        file.write(response_from_model)


/Users/hritik.raj/codegen-nai/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/hritik.raj/codegen-nai/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/hritik.raj/codegen-nai/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.w